In [2]:
import streamlit as st
import pandas as pd
import psycopg2

In [58]:
server = 'mtnatega.postgres.database.azure.com'
database = 'postgres'
username = 'natega'
password = 'Mt100323'
port = '5432'
sslmode = 'require'
conn = psycopg2.connect(
host=server,
database=database,
user=username,
password=password,
port=port,
sslmode=sslmode
)
cur = conn.cursor()
select_query = '''
    SELECT * FROM form
    LIMIT 1
    '''
cur.execute(select_query)
df = pd.DataFrame(cur.fetchall(), columns= [i[0]for i in cur.description])
df['id_image'].apply(lambda x: x.tobytes())
df['pay_image'].apply(lambda x: x.tobytes())

0    b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...
Name: pay_image, dtype: object

In [53]:
df

,no,name,dep,id,phone,pay_no,add_number,id_image_name,id_image,pay_image_name,pay_image
0,2,مصطفى سيد ابراهيم محمد,كهرباء,30104102200319,01154937978,01125671480,مرافق واحد (100 ج),IMG-20240731-WA0000.jpg,"[b'\xff', b'\xd8', b'\xff', b'\xe0', b'\x00', ...",IMG-20240802-WA0046.jpg,"[b'\xff', b'\xd8', b'\xff', b'\xe0', b'\x00', ..."


In [70]:
tuple(df['add_number'].value_counts())

(1,)

In [3]:


server = 'mtnatega.postgres.database.azure.com'
database = 'postgres'
username = 'natega'
password = 'Mt100323'
port = '5432'
sslmode = 'require'
conn = psycopg2.connect(
    host=server,
    database=database,
    user=username,
    password=password,
    port=port,
    sslmode=sslmode
)
cur = conn.cursor()
select_query = '''
    SELECT count(*) FROM form
    '''
cur.execute(select_query)
print(cur.fetchone())
conn.commit()
cur.close()
conn.close()

(129,)


In [20]:
import psycopg2

server = 'mtnatega.postgres.database.azure.com'
database = 'postgres'
username = 'natega'
password = 'Mt100323'
port = '5432'
sslmode = 'require'
conn = psycopg2.connect(
    host=server,
    database=database,
    user=username,
    password=password,
    port=port,
    sslmode=sslmode
)
cur = conn.cursor()
select_query = '''
ALTER TABLE form
ALTER COLUMN add_number TYPE VARCHAR(25);
'''
cur.execute(select_query)
conn.commit()
cur.close()
conn.close()

In [105]:
%%writefile form.py
import pandas as pd
import streamlit as st
import numpy as np
import psycopg2
from streamlit_extras.stylable_container import stylable_container
from functools import reduce

if 'server' not in globals():
    server = 'mtnatega.postgres.database.azure.com'
    database = 'postgres'
    username = 'natega'
    password = 'Mt100323'
    port = '5432'
    sslmode = 'require'

@st.cache_data()
def data():
    conn = psycopg2.connect(
    host=server,
    database=database,
    user=username,
    password=password,
    port=port,
    sslmode=sslmode
    )
    cur = conn.cursor()
    select_query = '''
        SELECT * FROM form
        '''
    cur.execute(select_query)
    df = pd.DataFrame(cur.fetchall(), columns= [i[0]for i in cur.description])
    return df['id_image'].apply(lambda x: x.tobytes()), df['pay_image'].apply(lambda x: x.tobytes()), df.drop(['id_image', 'pay_image', 'no'], axis = 1)
id_images, pay_images, df = data()
col1, col2 = st.columns([1,1],vertical_alignment='center')
with col1:
    with stylable_container(
        key="Upload_Data",
        css_styles="""
        h2{
        display: flex;
            justify-content: flex-end;
        }
        """
    ):
        st.header(f'عدد الطلاب المسجلين: {df.shape[0]}', divider='rainbow')
with col2:
    with stylable_container(
        key="Upload_Data",
        css_styles="""
        h2{
        display: flex;
            justify-content: flex-end;
        }
        """
    ):
        st.header(f'عدد المرافقين المسجلين: {sum(i*j for i, j in enumerate(df['add_number'].value_counts(), start = 1))}', divider='rainbow')
col1, col2 = st.columns([1, 1])
with col1:
    st.text_input('الاسم بالعربي رباعي', key='name', label_visibility='collapsed', placeholder="الاسم بالعربي رباعي")
with col2:
    st.selectbox('القسم', key='dep', options=[None, 'ميكانيكا صناعية', 'ميكاترونيكس', 'اتصالات', 'كهرباء', 'حاسبات', 'ميكانيكا انتاج', 'اتصالات credit', 'باور credit', 'طبية'], label_visibility='collapsed', placeholder="القسم")
col1, col2, col3 = st.columns([1, 1, 1])
with col1:
    st.text_input('الرقم القومي', key='ID', label_visibility='collapsed', placeholder="الرقم القومي", max_chars = 14)
with col2:
    st.text_input('رقم الهاتف', key='phone', label_visibility='collapsed', placeholder="رقم الهاتف")
with col3:
    st.selectbox('عدد المرافقين', key='add_no', options=[None, 'مرافق واحد (100 ج)', 'اتنين مرافقين (200 ج)', 'ثلاث مرافقين (300 ج)'], label_visibility='collapsed', placeholder="عدد المرافقين")

temp = df
if st.session_state['name'] != None and st.session_state['name'] != '':
    temp = temp[temp['name'].str.startswith(st.session_state['name'])]
if st.session_state['dep'] != None and st.session_state['dep'] != '':
    temp = temp[temp['dep'] == st.session_state['dep']]
if st.session_state['ID'] != None and st.session_state['ID'] != '':
    temp = temp[temp['id'] == st.session_state['ID']]
if st.session_state['phone'] != None and st.session_state['phone'] != '':
    temp = temp[temp['phone'].str.contains(st.session_state['phone'])]
if st.session_state['add_no'] != None and st.session_state['add_no'] != '':
    temp = temp[temp['add_number'] == st.session_state['add_no']]

for i, j in temp.iterrows():
    col1, col2 = st.columns([1,1],vertical_alignment='center')
    with col1:
        with stylable_container(
            key="Upload_Data",
            css_styles="""
            h2{
            display: flex;
                justify-content: flex-end;
            }
            """
        ):
            st.image(id_images[i], caption='صورة البطاقة')
    with col2:
        with stylable_container(
            key="Upload_Data",
            css_styles="""
            h2{
            display: flex;
                justify-content: flex-end;
            }
            """
        ):
            st.image(pay_images[i], caption='صورة عملية الدفع')
            with stylable_container(
                key="Upload_Data",
                css_styles="""
                p{
                    display: flex;
                    justify-content: flex-end;
                }
                """
            ):
                st.write(f'الاسم: {j['name']}')
                st.write(f'القسم: {j['dep']}')
                st.write(f'الرقم القومي: {j['id']}')
                st.write(f'رقم الهاتف: {j['phone']}')
                st.write(f'رقم العملية: {j['pay_no']}')
                st.write(f'عدد المرافقين : {j['add_number']}')

Overwriting form.py


In [16]:
conn = psycopg2.connect(
    host=server,
    database=database,
    user=username,
    password=password,
    port=port,
    sslmode=sslmode
)
cur = conn.cursor()
select_query = '''
SELECT id, name, add_number, phone, pay_no FROM form
WHERE id = '30101282103732'
'''
cur.execute(select_query)
pd.DataFrame(cur.fetchall())

,0,1,2,3,4
0,30101282103732,اسلام مدحت كامل سليمان,مرافق واحد (100 ج),01112342484,007600497481


In [30]:
import psycopg2

server = 'mtnatega.postgres.database.azure.com'
database = 'postgres'
username = 'natega'
password = 'Mt100323'
port = '5432'
sslmode = 'require'
conn = psycopg2.connect(
    host=server,
    database=database,
    user=username,
    password=password,
    port=port,
    sslmode=sslmode
)
cur = conn.cursor()
select_query = '''
SELECT name FROM form
'''
cur.execute(select_query)
data = cur.fetchall()

In [83]:
import os, shutil

try:
    directory_path = 'ID/'
    shutil.rmtree(directory_path)
except Exception as e:
    directory_path = 'ID/'
    os.makedirs(directory_path, exist_ok=True)
try:
    directory_path = './PAY/'
    shutil.rmtree(directory_path)
except Exception as e:
    directory_path = './PAY/'
    os.makedirs(directory_path, exist_ok=True)


In [84]:
for row in data:
    no, name, dep, id, phone, pay_no, add_no, id_image_name, id_image, pay_image_name, pay_image = row
    # Convert memoryview to bytes
    id_image_bytes = id_image.tobytes() if isinstance(id_image, memoryview) else id_image
    pay_image_bytes = pay_image.tobytes() if isinstance(pay_image, memoryview) else pay_image
    
    # Save ID image
    with open("./ID/"+name+'_'+id+ '_id' + id_image_name[id_image_name.rfind('.'):], 'wb') as id_image_file:
        id_image_file.write(id_image_bytes)
    
    # Save Pay image
    with open("./PAY/"+name+'_'+id+ '_pay' + pay_image_name[pay_image_name.rfind('.'):], 'wb') as pay_image_file:
        pay_image_file.write(pay_image_bytes)

In [85]:
pd.DataFrame(data).drop([0, 7, 8, 9, 10], axis = 1).rename({1:'الاسم', 2:'القسم', 3:'الرقم القومي', 4:'رقم الهاتف', 5:'رقم عملية التحويل', 6:'عدد المرافقين'}, axis = 1).to_excel('temp.xlsx')

In [30]:
pd.DataFrame(data, columns=['no', 'name', 'dep', 'id', 'phone', 'pay_no',
                            'add_no', 'id_image_name', 'id_image', 'pay_image_name', 'pay_image']).drop(['no', 'id_image_name', 'id_image', 'pay_image_name', 'pay_image'], axis = 1).to_excel('temp.xlsx')

In [31]:
pd.DataFrame(data, columns=['name']).to_excel('temp_name_add.xlsx')

In [21]:
cnx = psycopg2.connect(user="natega", password="{your_password}", host="mtnatega.postgres.database.azure.com", port=5432, database="postgres")
server = 'mtnatega.postgres.database.azure.com'
database = 'postgres'
username = 'natega'
password = 'Mt100323'
port = '5432'
sslmode = 'require'
conn = psycopg2.connect(
    host=server,
    database=database,
    user=username,
    password=password,
    port=port,
    sslmode=sslmode
)
cur = conn.cursor()
select_query = '''
DELETE FROM form
where id = '30101282103732'
'''
cur.execute(select_query)
conn.commit()